# 2. ¿Cuán rápido puedes ordenar estos productos?

## Descripción
La seccion de ofertas https://www.mercadolibre.com.ar/ofertas#nav-header es una sección que agrupa las mejores ofertas de MELI, y a grandes rasgos es un listado de productos en oferta ordenados por un score de ML y distintas reglas de negocio. Anexado al desafío, se encuentra un archivo “ordenamiento.csv” el cual tiene un listado de productos, con su score y categorías para este desafío.
El score determina cuán bueno es un item_id, siendo 1 el mejor valor posible.
El problema es que este ordenamiento debe reflejar variedad de productos y categorías permitiendo el discovery de distintos tipos de productos por lo cual tenemos en producción reglas como las siguientes:
1. El “domain_id” no se puede repetir en 4 posiciones consecutivas.
2. El “vertical” no se puede repetir en 1 posición consecutiva.
3. De existir el id 641416750 en el listado debe estar en la posición 3 siendo esta regla más fuerte que las demás.
4. De existir el id 22351223 en el listado debe estar en la posición 6 siendo esta regla más fuerte que las demás.
5. Las posiciones 9,10,11 deben tener sí o sí items de la categoría “HOME&DECOR” siendo esta regla más fuerte que la 1 y 2.
6. Cumpliendo estas condiciones, el ordenamiento debe respetar un ordenamiento de mayor score a menor.
El desafío es diseñar un algoritmo que, dado el dataset y estas restricciones, devuelva el listado final ordenado de ítems. El algoritmo debe estar diseñado para escalar eficientemente con el número de ítems, y contemplar los casos en que no se pueden cumplir las restricciones. **¡El tiempo de ejecución es el factor clave!**

## Entregable
Notebook con el algoritmo para generar el listado ordenado y su tiempo de ejecución.

## Desarrollo del código

### 1. Importe de librerias necesarias para la transformación del dataset
- **pandas** para el manejo del dataset como dataframe.
- **numpy** para cálculos numéricos y operaciones con matrices y arreglos.
- **time** para contabilizar el tiempo de ejecución del código

In [1]:
import pandas as pd
import numpy as np
import time

### 2. Inicio del tiempo de ejecución del código
Con la función de time de la librería time empieza un contador a correr hasta que llegue al comando que le dice que pare

In [2]:
# Registra el tiempo de inicio
inicio = time.time()

### 3. Lectura del dataset de ordenamiento

In [3]:
df = pd.read_csv('ordenamiento.csv')

### 4. Creación de la regla 3 y 4
Por lo visto, algunas reglas tiene prioridad sobre otras y este el es caso de las reglas 3 y 4. Estas dicen que en caso de existir cierto numero de item, deben ir en posiciones específicas y que estas reglas en especial tienen prioridad sobre las demás. En este caso la función **move_record_to_position()** tiene como objetivo verificar si el item existe y de ser así moverlo a la posición indicada; de tal forma los valores de entrada corresponde al dataframe, el id del item a mover y la posición deseada en la cual colocar dicho item. La función devuelve el mismo dataframe pero ordenado con esta regla.

In [4]:
def move_record_to_position(df, id_item_a_mover, posicion_deseada):
    # Verificar si el 'id_item_a_mover' existe en el DataFrame
    if id_item_a_mover not in df['item_id'].values:
        #print(f"El id {id_item_a_mover} no existe en el DataFrame.")
        return df
    
    # Obtener el registro a mover y su índice actual
    idx = df[df['item_id'] == id_item_a_mover].index.item()
    registro_a_mover = df.iloc[idx]

    # Eliminar el registro del DataFrame original
    df = df.drop(idx)

    # Insertar el registro en la posición deseada dentro del DataFrame
    df = pd.DataFrame(np.insert(df.values, posicion_deseada, registro_a_mover.values, axis=0), columns=df.columns)

    return df

### 5. Creación de la regla 5
En el caso de esta regla, la función **replace_records_in_position()** cumple con exactamente la descripción de la regla. En ese sentido, toma la columna a filtrar (en este caso *category*) con el valor de la categoría requerida (en este caso *“HOME&DECO”*), ordena y obtiene el top 3 de esa categoría para ser reemplazado directamente en las posiciones 9,10,11 del dataframe respetando el orden con el que ya venía.

In [5]:
def replace_records_in_position(df, column_name, value_to_filter):
    # Filtrar el DataFrame por el valor especificado en la columna
    filtered_df = df[df[column_name] == value_to_filter].copy()

    # Convertir la columna 'score' a tipo de datos numérico (si no lo es ya)
    filtered_df['score'] = pd.to_numeric(filtered_df['score'])

    # Obtener los tres registros con la puntuación más alta
    top_records = filtered_df.nlargest(3, 'score')

    # Encontrar las posiciones de los registros del top 3 en el DataFrame original
    top_positions = df[df['item_id'].isin(top_records['item_id'])].index

    # Reemplazar los registros en las posiciones 9, 10 y 11 del DataFrame original
    for i, top_record in enumerate(top_records.values):
        source_index = top_positions[i]
        dest_index = 9 + i

        if dest_index >= source_index:
            dest_index += 1

        # Desplazar las filas hacia abajo desde dest_index hasta source_index
        df.iloc[dest_index+1:source_index+1] = df.iloc[dest_index:source_index].values

        # Reemplazar el registro en dest_index con el registro del top 3
        df.iloc[dest_index] = top_record

    return df

### 6. Creación de la regla 1 y 2
En el caso de la regla 1 y 2, se deben tener en cuenta las veces en las que se repite un valor en el ordenamiento y al ser reglas tan similares, cree la función **sort_dataframe_with_restriction()** que tiene en cuenta ambas restricciones de manera que:
- df: El dataframe que se necesita ordenar
- field_restriction1: en este caso funciona para revisar el campo domain
- n_times1: en este caso funciona para revisar las veces que no se puede repetir el campo 1 (en este caso domain)
- field_restriction2: en este caso funciona para revisar el campo vertical
- n_times2: en este caso funciona para revisar las veces que no se puede repetir el campo 2 (en este caso vertical)

In [6]:
# Función para ordenar el DataFrame con las restricciones de domain_id y otro campo adicional
def sort_dataframe_with_restriction(df, field_restriction1, n_times1, field_restriction2, n_times2):
    # Copiar el DataFrame para no modificar el original
    df_sorted = df.copy()

    # Ordenar por la columna "score" de manera descendente
    df_sorted = df_sorted.sort_values(by='score', ascending=False)

    # Verificar ambas restricciones al mismo tiempo
    consecutive_repeats1, consecutive_repeats2 = 0, 0
    prev_field_value1, prev_field_value2 = None, None
    for i, row in df_sorted.iterrows():
        if row[field_restriction1] == prev_field_value1:
            consecutive_repeats1 += 1
            if consecutive_repeats1 < n_times1:
                # Si se excede la restricción, mover el registro a una posición anterior
                if i - 1 >= 0:
                    df_sorted = df_sorted.drop(i)
                    df_sorted = pd.concat([df_sorted.iloc[:i-1], row.to_frame().T, df_sorted.iloc[i-1:]])
        else:
            consecutive_repeats1 = 0
        prev_field_value1 = row[field_restriction1]

        if row[field_restriction2] == prev_field_value2:
            consecutive_repeats2 += 1
            if consecutive_repeats2 < n_times2:
                # Si se excede la restricción, mover el registro a una posición anterior
                if i - 1 >= 0:
                    df_sorted = df_sorted.drop(i)
                    df_sorted = pd.concat([df_sorted.iloc[:i-1], row.to_frame().T, df_sorted.iloc[i-1:]])
        else:
            consecutive_repeats2 = 0
        prev_field_value2 = row[field_restriction2]

    # Reiniciar el índice del DataFrame después de las modificaciones
    df_sorted = df_sorted.reset_index(drop=True)

    return df_sorted

### 7. Ordenar el DataFrame con las restricciones
Aplico las funciones en el orden requerido teniendo en cuenta que:
1. La función sort_dataframe_with_restriction() ordena inmediatamente el dataframe con las reglas 1 y 2.
2. Aplico la función move_record_to_position() ya que deseo mantener esos id en dichas posiciones respetando el ordenamiento previo, tal como dice la regla 3 y 4.
3. Por último dejo la función replace_records_in_position() porque deseo mover el ordenamiento a partir de la posición 9 con las reglas anteriores con el top 3 de la columna y el campo requerido, siguiendo con la regla 5.
De esta forma garantizo la prioridad de las reglas en el siguiente orden:
- 3 -> 4 -> 5 -> 1 -> 2 -> 6

In [7]:
df_sorted_with_restriction = sort_dataframe_with_restriction(df, 'domain', 4, 'vertical', 1)
df_sorted_with_restriction = move_record_to_position(df_sorted_with_restriction, 641416750, 3)
df_sorted_with_restriction = move_record_to_position(df_sorted_with_restriction, 22351223, 6)
df_resultado = replace_records_in_position(df_sorted_with_restriction, 'category', 'HOME&DECOR')

### 8. Termina el cálculo de tiempo
Envía stop al timer

In [8]:
# Registra el tiempo de finalización
fin = time.time()

### 9. Muestro el tiempo final de ejecución

In [9]:
# Calcula el tiempo total de ejecución
tiempo_total = fin - inicio
print("Tiempo total de ejecución:", tiempo_total, "segundos")

Tiempo total de ejecución: 0.4264833927154541 segundos


### 10. Guardo el dataframe ordenado

In [10]:
df_resultado.to_csv('df_resultado.csv', index=False)